In [2]:
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import pandas as pd

In [1]:
%matplotlib notebook

In [ ]:
!pip install tables

In [165]:
#!pip install --quiet nma-ibl

import datajoint as dj

dj.config['database.host'] = 'datajoint-public.internationalbrainlab.org'
dj.config['database.user'] = 'ibl-public'
dj.config['database.password'] = 'ibl-public'
dj.conn() # explicitly checks if database connection can be established

[2023-07-21 13:53:44,804][INFO]: Connecting ibl-public@datajoint-public.internationalbrainlab.org:3306
[2023-07-21 13:53:46,189][INFO]: Connected ibl-public@datajoint-public.internationalbrainlab.org:3306


DataJoint connection (connected) ibl-public@datajoint-public.internationalbrainlab.org:3306

In [166]:
from nma_ibl import reference, subject, action, acquisition, data, behavior, behavior_analyses

In [ ]:
#--------Get fully trained subject uuids and get associated data------
subject_training = behavior_analyses.SessionTrainingStatus.fetch(format="frame").reset_index().groupby("subject_uuid").training_status.value_counts().unstack()
fully_trained_subjects = subject_training.loc[subject_training.ready4ephysrig > 0].index
fully_trained_data = (behavior.TrialSet.Trial  * acquisition.Session * subject.Subject & [{'subject_uuid': uuid} for uuid in fully_trained_subjects]).fetch(format='frame')


In [ ]:
#-------Save fully trained mouse data
fully_trained_data.to_hdf(r"D:\fully_trained_data.h5", "df")

In [104]:
#-------Load saved data
fully_trained_data = pd.read_hdf(r"D:\fully_trained_data.h5")


In [105]:
fully_trained_data.reset_index(inplace = True)
fully_trained_data['subject_uuid'] = fully_trained_data['subject_uuid'].astype('string')
fully_trained_data['session_uuid'] = fully_trained_data['session_uuid'].astype('string')
fully_trained_data['signed_contrast'] = fully_trained_data['trial_stim_contrast_right'] - fully_trained_data['trial_stim_contrast_left']
fully_trained_data['report_right'] = fully_trained_data['trial_response_choice'] == "CCW"


In [106]:

n_sessions = fully_trained_data.session_uuid.unique().shape[0]
n_trials = fully_trained_data.shape[0]
n_subjects = fully_trained_data.subject_uuid.unique().shape[0]

print("{} trials, {} sessions, {} subjects for fully trained mice".format(n_trials, n_sessions, n_subjects))


3814816 trials, 5485 sessions, 100 subjects for fully trained mice


In [107]:
fully_trained_data.columns

Index(['subject_uuid', 'session_start_time', 'trial_id', 'trial_start_time',
       'trial_end_time', 'trial_response_time', 'trial_response_choice',
       'trial_stim_on_time', 'trial_stim_contrast_left',
       'trial_stim_contrast_right', 'trial_feedback_time',
       'trial_feedback_type', 'trial_rep_num', 'trial_go_cue_time',
       'trial_go_cue_trigger_time', 'trial_stim_prob_left',
       'trial_reward_volume', 'trial_iti_duration', 'trial_included',
       'session_uuid', 'session_number', 'session_end_time', 'session_lab',
       'session_location', 'task_protocol', 'session_type',
       'session_narrative', 'session_ts', 'subject_nickname', 'sex',
       'subject_birth_date', 'ear_mark', 'subject_line', 'subject_strain',
       'subject_source', 'protocol_number', 'subject_description',
       'subject_ts', 'signed_contrast', 'report_right'],
      dtype='object')

In [110]:
# check for duplicates - no duplication if using session start time
for ncolumn in range(fully_trained_data.columns.shape[0]):
    cols = fully_trained_data.columns[:ncolumn+1].tolist()
    print(cols)
    duplicated = fully_trained_data.duplicated(cols).any()
    print(duplicated)
    if duplicated == False:
        break


['subject_uuid']
True
['subject_uuid', 'session_start_time']
True
['subject_uuid', 'session_start_time', 'trial_id']
False


In [115]:
duplicated = fully_trained_data.duplicated(["subject_uuid", "session_uuid", "trial_id"]).any()
print(duplicated)

True


In [114]:
# look at first and last duplicates
first = fully_trained_data[fully_trained_data.duplicated(["subject_uuid", "session_uuid", "trial_id"], keep = "first")]
last = fully_trained_data[fully_trained_data.duplicated(["subject_uuid", "session_uuid",  "trial_id"], keep = "last")]

In [137]:
matches  =first.reset_index() == last.reset_index()
matches.all()

index                        False
subject_uuid                  True
session_start_time           False
trial_id                      True
trial_start_time              True
trial_end_time               False
trial_response_time          False
trial_response_choice         True
trial_stim_on_time           False
trial_stim_contrast_left      True
trial_stim_contrast_right     True
trial_feedback_time          False
trial_feedback_type           True
trial_rep_num                False
trial_go_cue_time            False
trial_go_cue_trigger_time    False
trial_stim_prob_left          True
trial_reward_volume          False
trial_iti_duration           False
trial_included               False
session_uuid                  True
session_number                True
session_end_time             False
session_lab                   True
session_location              True
task_protocol                 True
session_type                  True
session_narrative            False
session_ts          

In [154]:
mismatched_cols = matches.all()[~matches.all()].index
mismatched_cols

Index(['index', 'session_start_time', 'trial_end_time', 'trial_response_time',
       'trial_stim_on_time', 'trial_feedback_time', 'trial_rep_num',
       'trial_go_cue_time', 'trial_go_cue_trigger_time', 'trial_reward_volume',
       'trial_iti_duration', 'trial_included', 'session_end_time',
       'session_narrative', 'session_ts', 'ear_mark', 'subject_line',
       'subject_strain', 'subject_source', 'subject_description'],
      dtype='object')

In [164]:
## duplicated have session_start time out by 100th of a second 
## last duplicated have trial response time to more decimal places
## duplicated differ on session_ts

#-----conclusion ALWAYS KEEP FIRST!!!-------

for mismatched_col in mismatched_cols:
    if mismatched_col in first.columns:
        trials_mismatched = np.where(first[mismatched_col].to_numpy()!= last[mismatched_col].to_numpy())[0]
        print(mismatched_col, len(trials_mismatched), first[mismatched_col].iloc[trials_mismatched].head(), last[mismatched_col].iloc[trials_mismatched].head())


session_start_time 34356 224845   2020-02-20 15:29:44
224846   2020-02-20 15:29:44
224847   2020-02-20 15:29:44
224848   2020-02-20 15:29:44
224849   2020-02-20 15:29:44
Name: session_start_time, dtype: datetime64[ns] 224495   2020-02-20 15:29:43
224496   2020-02-20 15:29:43
224497   2020-02-20 15:29:43
224498   2020-02-20 15:29:43
224499   2020-02-20 15:29:43
Name: session_start_time, dtype: datetime64[ns]
trial_end_time 994 2644779    15.290038
2644780    18.245475
2644781    23.758224
2644782    29.634803
2644783    33.624837
Name: trial_end_time, dtype: float64 2643785    15.790037
2643786    18.745475
2643787    24.258224
2643788    30.134798
2643789    34.124837
Name: trial_end_time, dtype: float64
trial_response_time 33647 224845     80.3317
224846     85.7852
224847     90.0733
224848     96.6324
224849    100.1300
Name: trial_response_time, dtype: float64 224495     80.331707
224496     85.785243
224497     90.073255
224498     96.632414
224499    100.129856
Name: trial_respon

In [167]:
#------drop duplicates-----
fully_trained_data.drop_duplicates(["subject_uuid", "session_uuid", "trial_id"], keep = "first", inplace = True)

In [168]:
fully_trained_data.columns

Index(['subject_uuid', 'session_start_time', 'trial_id', 'trial_start_time',
       'trial_end_time', 'trial_response_time', 'trial_response_choice',
       'trial_stim_on_time', 'trial_stim_contrast_left',
       'trial_stim_contrast_right', 'trial_feedback_time',
       'trial_feedback_type', 'trial_rep_num', 'trial_go_cue_time',
       'trial_go_cue_trigger_time', 'trial_stim_prob_left',
       'trial_reward_volume', 'trial_iti_duration', 'trial_included',
       'session_uuid', 'session_number', 'session_end_time', 'session_lab',
       'session_location', 'task_protocol', 'session_type',
       'session_narrative', 'session_ts', 'subject_nickname', 'sex',
       'subject_birth_date', 'ear_mark', 'subject_line', 'subject_strain',
       'subject_source', 'protocol_number', 'subject_description',
       'subject_ts', 'signed_contrast', 'report_right'],
      dtype='object')

In [ ]:
# add response time column from trial
# calculate wheel dynamics
# measure facial dynamics - pupil dilation

In [180]:
COLS = ['subject_uuid', 'session_start_time', 'trial_id', 'trial_stim_prob_left',
        'trial_response_choice', 'trial_stim_contrast_right',
         'trial_stim_contrast_left', 'signed_contrast', "report_right",
         'trial_reward_volume'
         ]

In [181]:
data_df = fully_trained_data[COLS].copy()
data_df.size/1e9

0.0378046

In [182]:
data_df.to_csv(r"D:\ibl_dataframe_fully_trained_mice.csv")
data_df.head()

,subject_uuid,session_start_time,trial_id,trial_stim_prob_left,trial_response_choice,trial_stim_contrast_right,trial_stim_contrast_left,signed_contrast,report_right,trial_reward_volume
0,019a22c1-b944-4494-9e38-0e45ae6697bf,2019-08-27 14:25:30,1,0.5,CCW,0.5,0.0,0.5,True,NaN
1,019a22c1-b944-4494-9e38-0e45ae6697bf,2019-08-27 14:25:30,2,0.5,CCW,0.0,1.0,-1.0,True,NaN
2,019a22c1-b944-4494-9e38-0e45ae6697bf,2019-08-27 14:25:30,3,1.0,CCW,0.0,1.0,-1.0,True,NaN
3,019a22c1-b944-4494-9e38-0e45ae6697bf,2019-08-27 14:25:30,4,1.0,CCW,1.0,0.0,1.0,True,NaN
4,019a22c1-b944-4494-9e38-0e45ae6697bf,2019-08-27 14:25:30,5,0.5,CW,0.0,0.5,-0.5,False,NaN


In [177]:
data_df.shape

(3780460, 9)

In [49]:
fully_trained_data[fully_trained_data.trial_included.isna()].shape

(3814816, 40)

In [57]:
fully_trained_data[fully_trained_data.trial_included.isnull()].trial_included.unique()

array([None], dtype=object)

In [45]:
pd.DataFrame([first.iloc[0], last.iloc[0]]).T

,224845,224495
subject_uuid,0f47b24d-3909-428c-a18d-666006c7c834,0f47b24d-3909-428c-a18d-666006c7c834
session_start_time,2020-02-20 15:29:44,2020-02-20 15:29:43
trial_id,1,1
trial_start_time,74.568961,74.568961
trial_end_time,81.904421,81.904421
trial_response_time,80.3317,80.331707
trial_response_choice,CCW,CCW
trial_stim_on_time,79.488404,79.488404
trial_stim_contrast_left,0.0,0.0
trial_stim_contrast_right,1.0,1.0


In [36]:
first.reset_index() == last.reset_index()

,index,subject_uuid,session_start_time,trial_id,trial_start_time,trial_end_time,trial_response_time,trial_response_choice,trial_stim_on_time,trial_stim_contrast_left,...,subject_birth_date,ear_mark,subject_line,subject_strain,subject_source,protocol_number,subject_description,subject_ts,signed_contrast,report_right
0,False,True,False,True,True,True,False,True,True,True,...,True,True,False,False,False,True,False,True,True,True
1,False,True,False,True,True,True,False,True,True,True,...,True,True,False,False,False,True,False,True,True,True
2,False,True,False,True,True,True,False,True,True,True,...,True,True,False,False,False,True,False,True,True,True
3,False,True,False,True,True,True,False,True,True,True,...,True,True,False,False,False,True,False,True,True,True
4,False,True,False,True,True,True,False,True,True,True,...,True,True,False,False,False,True,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34351,False,True,False,True,True,True,False,True,True,True,...,True,True,True,False,True,True,True,True,True,True
34352,False,True,False,True,True,True,False,True,True,True,...,True,True,True,False,True,True,True,True,True,True
34353,False,True,False,True,True,True,False,True,True,True,...,True,True,True,False,True,True,True,True,True,True
34354,False,True,False,True,True,True,False,True,True,True,...,True,True,True,False,True,True,True,True,True,True


In [37]:
first.session_start_time

224845    2020-02-20 15:29:44
224846    2020-02-20 15:29:44
224847    2020-02-20 15:29:44
224848    2020-02-20 15:29:44
224849    2020-02-20 15:29:44
                  ...        
3014070   2019-10-07 10:44:18
3014071   2019-10-07 10:44:18
3014072   2019-10-07 10:44:18
3014073   2019-10-07 10:44:18
3014074   2019-10-07 10:44:18
Name: session_start_time, Length: 34356, dtype: datetime64[ns]

In [38]:
last.session_start_time

224495    2020-02-20 15:29:43
224496    2020-02-20 15:29:43
224497    2020-02-20 15:29:43
224498    2020-02-20 15:29:43
224499    2020-02-20 15:29:43
                  ...        
3013240   2019-10-07 10:44:17
3013241   2019-10-07 10:44:17
3013242   2019-10-07 10:44:17
3013243   2019-10-07 10:44:17
3013244   2019-10-07 10:44:17
Name: session_start_time, Length: 34356, dtype: datetime64[ns]

In [24]:
# if so
fully_trained_data[fully_trained_data.duplicated(["subject_uuid", "session_uuid", "trial_start_time", "trial_id", "trial_stim_contrast_left"], keep = False)]

,subject_uuid,session_start_time,trial_id,trial_start_time,trial_end_time,trial_response_time,trial_response_choice,trial_stim_on_time,trial_stim_contrast_left,trial_stim_contrast_right,...,subject_birth_date,ear_mark,subject_line,subject_strain,subject_source,protocol_number,subject_description,subject_ts,signed_contrast,report_right
224495,0f47b24d-3909-428c-a18d-666006c7c834,2020-02-20 15:29:43,1,74.568961,81.904421,80.331707,CCW,79.488404,0.0000,1.000,...,2019-07-23,0,None,None,None,1,None,2020-01-13 23:39:07,1.0000,True
224496,0f47b24d-3909-428c-a18d-666006c7c834,2020-02-20 15:29:43,2,82.322623,88.357860,85.785243,CCW,82.844001,0.2500,0.000,...,2019-07-23,0,None,None,None,1,None,2020-01-13 23:39:07,-0.2500,True
224497,0f47b24d-3909-428c-a18d-666006c7c834,2020-02-20 15:29:43,3,88.799960,91.656952,90.073255,CCW,89.293882,0.0000,0.250,...,2019-07-23,0,None,None,None,1,None,2020-01-13 23:39:07,0.2500,True
224498,0f47b24d-3909-428c-a18d-666006c7c834,2020-02-20 15:29:43,4,92.112548,98.193538,96.632414,CCW,95.593810,0.0000,0.125,...,2019-07-23,0,None,None,None,1,None,2020-01-13 23:39:07,0.1250,True
224499,0f47b24d-3909-428c-a18d-666006c7c834,2020-02-20 15:29:43,5,98.646142,101.693573,100.129856,CCW,99.639362,0.0000,0.250,...,2019-07-23,0,None,None,None,1,None,2020-01-13 23:39:07,0.2500,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3014070,cb21532d-eb0f-4706-b088-bf4afc65e81a,2019-10-07 10:44:18,826,4055.105799,4057.687100,4056.190000,CCW,4055.558199,0.0000,0.250,...,2019-06-10,RF,C57BL/6J,None,Charles River,1,Blinded experiment.,2020-01-13 23:40:10,0.2500,True
3014071,cb21532d-eb0f-4706-b088-bf4afc65e81a,2019-10-07 10:44:18,827,4058.917499,4067.078701,4065.580000,CW,4059.627599,0.0625,0.000,...,2019-06-10,RF,C57BL/6J,None,Charles River,1,Blinded experiment.,2020-01-13 23:40:10,-0.0625,False
3014072,cb21532d-eb0f-4706-b088-bf4afc65e81a,2019-10-07 10:44:18,828,4068.410099,4070.846001,4069.350000,CCW,4068.877099,0.0000,0.125,...,2019-06-10,RF,C57BL/6J,None,Charles River,1,Blinded experiment.,2020-01-13 23:40:10,0.1250,True
3014073,cb21532d-eb0f-4706-b088-bf4afc65e81a,2019-10-07 10:44:18,829,4072.085299,4087.578701,4085.080000,CW,4072.640999,0.0000,0.000,...,2019-06-10,RF,C57BL/6J,None,Charles River,1,Blinded experiment.,2020-01-13 23:40:10,0.0000,False
